In [ ]:
!unzip -o gene_prediction_data.zip > /dev/null

In [ ]:
#@title Provide information about the gene

Essential = 'Ambiguous-Essential' #@param ['Non-Essential', 'Ambiguous-Essential', '?', 'Essential']
Class = 'Ubiquitin-system proteins' #@param['Histones', 'Peptidyl-Prolyl-Isomerases', 'Protein Kinases', '?', 'Major facilitator superfamily proteins (MFS)', 'Protein phosphatases', 'Polymerases', 'GTP/GDP-exchange factors (GEFs)', 'Other kinases', 'Ubiquitin-system proteins', 'GTP/GDP dissociation inhibitors (GDIs)', 'Transcription factors', 'Nucleases', 'Motorproteins', 'Cyclins', 'Adaptins', 'Molecular chaperones', 'Tubulins', 'ATPases', 'Unknown', 'Proteases', 'GTPase activating proteins (GAPs)', 'Actin related proteins', 'GTP-binding proteins']
Complex = 'Transcription complexes/Transcriptosome' #@param['Glucan synthases', 'Serine/threonine phoshpoprotein phosphatase', 'delta3-cis-delta2-trans-enoyl-CoA isomerase', 'Translation complexes', 'Arginase', 'Endonuclease SceI, mitochondrial', 'Proteases, mitochondrial', 'Sister chromatid cohesion regulation complex', 'Phosphofructokinase', '?', 'Replication complexes', 'Gim complexes', 'Histone acetyltransferase complexes', 'Alpha, alpha-trehalose-phosphate synthase', 'L-aminopadipate-semialdehyde dehydrogenase', 'Fatty acid synthetase, cytoplasmic', 'Pyruvate kinase', 'Transcription complexes/Transcriptosome', 'SCF (Skp1-Cdc53-F-box protein) complexes', 'Nonsense-mediated mRNA decay pathway complex', 'cAMP-dependent protein kinase', 'RSC complex (Remodel the structure of chromatin)', 'Casein kinase', 'Nucleosomal protein complex', 'Intracellular transport complexes', 'Prenyltransferases', 'Arginine-specific carbamoylphosphate synthase', 'Nuclear pore complex (NPC)', 'Assembly complexes', 'Kinetochore protein complexes', 'Translocon', 'Chaperonine containing T-complex TRiC (TCP RING Complex)', 'Synaptonemal complex (SC)', 'Ribonucleoside-diphosphate reductase', 'Kel1p/Kel2p complex', 'Pyruvate dehydrogenase', 'Succinyl-CoA ligase', 'Acetolactate synthase', '2-oxoglutarate dehydrogenase', 'Unknown', 'Anaphase promoting complex (APC)', 'Histone deacetylase complexes', 'Cytoskeleton', 'Proteasome', 'Cyclin-CDK (Cyclin-dependent kinases) complexes', 'RNA processing complexes', 'Mitochondrial translocase complex', 'Exocyst complex', 'Nem1p-Spo7p complex', 'H+-transporting ATPase, vacuolar', 'Respiration chain complexes', 'Calcineurin B']
Motif = 'PS00951' #@param {type:"string"}
Phenotype = 'Sensitivity to immunosuppressants' #@param['Conditional phenotypes', 'Nucleic acid metabolism defects', 'Cell morphology and organelle mutants', 'Sensitivity to aminoacid analogs and other drugs', 'Auxotrophies, carbon and', 'Carbohydrate and lipid biosynthesis defects', 'Sensitivity to antibiotics', 'Mating and sporulation defects', 'Sensitivity to immunosuppressants', 'Stress response defects', 'Cell cycle defects', '?']

Chromosome_number = 16 #@param {type:"slider", min:1, max:16, step:1}


### If you want to add information about gene interactions
### Press the run button on the next cell and Add the gene IDs of the genes which interact with the gene you provided,
### along with their interaction coefficient.
### example:
### G234565 : 0.87259026 <br>
### G235528 : 0.9679347



In [ ]:
#@title
print("Enter Interactions. Press Ctrl-D or Ctrl-Z  to save it.")
interactions = []
while True:
    try:
        line = input()
    except EOFError:
        break
    interactions.append(line)

Enter Interactions. Press Ctrl-D or Ctrl-Z  to save it.



In [ ]:
#@title
interactions_dict = {}
if(interactions_dict):
  for interaction in interactions:
    info = interaction.split(":")
    interactions_dict[info[0]] = info[1]

In [ ]:
#@title What do you want to predict

target = 'Function' #@param ['Localization', 'Function']

In [ ]:
#@title
import pandas as pd
import pickle


def predict_localization_no_interactions(candidate_gene_series):
  localization_model = 'gene_prediction_data/models/svm_popular_localization.pickle'
  svm_popular_localization = pickle.load(open(localization_model, "rb"))

  return svm_popular_localization.predict([candidate_gene])


def predict_function_no_interactions(candidate_gene_series):
  function_model = 'gene_prediction_data/models/svm_popular_function.pickle'
  svm_popular_function = pickle.load(open(function_model, "rb"))

  return svm_popular_function.predict([candidate_gene_series])


from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import jaccard_score

def custom_distance_interactions(a, b, **kwargs):

    similarity = 0

    interactions_dict = kwargs['interactions']

    gene_id_b = "G2" + str(int(b[-1]))

    # print(int(a[-1]))
    # print(gene_id)
    if(gene_id_b in interactions_dict.keys()):
            similarity += abs(float(interactions_dict[gene_id_b]) * kwargs["interactions_coefficient"])

    similarity += jaccard_score(a[:-1], b[:-1])

    return 2 - similarity

def predict_with_interactions(candidate_gene_series, target_feature='Localization'):
  target_localization = pd.read_csv("gene_prediction_data/target_localization.csv").iloc[: , 1:]
  target_function = pd.read_csv("gene_prediction_data/target_function.csv").iloc[: , 1:]


  all_data = pd.read_csv("gene_prediction_data/gene_data_most_popular.csv")

  IDs = all_data['Gene ID']
  X = all_data.drop('Gene ID', axis=1)
  X = X.append(candidate_gene_series, ignore_index=True)
  IDs = [float(ID.replace("G2", "")) for ID in IDs]
  IDs.append(0.0)

  X = pd.get_dummies(X, dtype=float)

  X['Gene ID'] = IDs

  candidate_gene = X.iloc[-1]

  X_train = X.iloc[0:4346]
  target_localization = target_localization.iloc[0:4346]
  target_function = target_function.iloc[0:4346]

  X_train.head()
  if(target_feature == 'Localization'):
    knn_localization = KNeighborsClassifier(metric=custom_distance_interactions, metric_params={"interactions":interactions_dict, "interactions_coefficient":1})
    knn_localization.fit(X_train, target_localization)
    return knn_localization.predict([candidate_gene])

  elif(target_feature == 'Function'):
    knn_function = KNeighborsClassifier(metric=custom_distance_interactions, metric_params={"interactions":interactions_dict, "interactions_coefficient":1})
    knn_function.fit(X_train, target_function)
    # return target_function
    return knn_function.predict([candidate_gene])

train_data = pd.read_csv("gene_prediction_data/gene_data_most_popular.csv").drop("Gene ID", axis=1)
candidate_gene_series = pd.Series(data=[Essential, Class, Complex, Phenotype, Motif, str(Chromosome_number)], index = ['Essential', 'Class', 'Complex', 'Phenotype', 'Motif', 'Chromosome Number'])

print(len(set(train_data['Chromosome Number'])))
print(set(train_data['Chromosome Number']))
## check if all the provided data exist in the training set
for indx, values in candidate_gene_series.items():
  if(values not in set(list(train_data[indx]))):
    candidate_gene_series[indx] = '?'

## append the datapoint to the larger dataset to get the right dummy variables
appended_data = train_data.append(candidate_gene_series, ignore_index=True)
dummies_data = pd.get_dummies(appended_data)
candidate_gene = dummies_data.iloc[-1]

if(len(interactions) == 0):
  if(target == 'Localization'):
    prediction = predict_localization_no_interactions(candidate_gene)
  elif(target == 'Function'):
    prediction = predict_function_no_interactions(candidate_gene)
else:
  if(target =='Localization'):
    prediction = predict_with_interactions(candidate_gene_series, target_feature='Localization')
  elif(target == 'Function'):
    prediction = predict_with_interactions(candidate_gene_series, target_feature='Function')


17
{'6', '7', '10', '8', '3', '9', '15', '4', '1', '11', '12', '14', '16', '5', '?', '2', '13'}


<ipython-input-15-8318e4b29a0f>:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_data = train_data.append(candidate_gene_series, ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
#@title
if(target == "Localization"):
  print("The most probable localization of the gene is:", end=" ")
elif(target == 'Function'):
  print("The most probable function of the gene is:", end=" ")

print(prediction[0])

The most probable function of the gene is: CELLULAR ORGANIZATION (proteins are localized to the corresponding organelle)
